# Verschieben der @n-Attribute aus den <milestone>-Elementen in <note>-Elemente

In [ ]:
from lxml import etree
import csv

input_path = "C:/Users/.../Schlegel-Briefe_zusammengefuehrt.xml"
output_path = "C:/Users/.../Schlegel-Briefe_milestone_transformiert.xml"
csv_log_path = "C:/Users/.../missing_notes_log.csv"

tree = etree.parse(input_path)
root = tree.getroot()

ns = {"tei": "http://www.tei-c.org/ns/1.0"}
missing_n_entries = []

for milestone in root.xpath("//tei:milestone[@n]", namespaces=ns):
    n_value = milestone.get("n")
    next_note = milestone.xpath("following::tei:note[1]", namespaces=ns)

    if next_note:
        note = next_note[0]
        # Kombiniere mehrere n-Werte, falls notwendig
        if note.get("n"):
            existing = note.get("n")
            if n_value not in existing.split(", "):  # Duplikate vermeiden
                note.set("n", f"{existing},{n_value}")
        else:
            note.set("n", n_value)
        milestone.attrib.pop("n")
    else:
        # Für Logging
        line = milestone.sourceline
        path = root.getroottree().getpath(milestone)
        missing_n_entries.append({"n": n_value, "line": line, "xpath": path})

tree.write(output_path, encoding="utf-8", xml_declaration=True, pretty_print=True)

with open(csv_log_path, mode="w", newline='', encoding="utf-8") as csv_file:
    fieldnames = ["n", "line", "xpath"]
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(missing_n_entries)

print(f"Fertig! {len(missing_n_entries)} milestones ohne note. Alle n-Werte kombiniert, nichts mehr überschrieben.")



Fertig! 0 milestones ohne note. Alle n-Werte kombiniert, nichts mehr überschrieben.
